In [1]:
import pandas as pd
import pickle
import numpy as np
import networkx as nx
import re
import time

In [15]:
# df_ivs = pickle.load(open('data/ivs_exploded_100.p', 'rb'))
# df_ivs_old = pickle.load(open('data/ivs_exploded_100_old.p', 'rb'))
# routes_dests = pickle.load(open('data/users_ship_specific_routes.p', 'rb'))
# df_ships = pd.read_excel('data/ship_types.xlsx')

# G = pickle.load(open("data/network.p", "rb"))
# paths = pickle.load(open("data/paths.p", "rb"))
# df_random = pickle.load(open("data/df_random.p", "rb"))
# df_abm = pickle.load(open("data/df_abm.p", "rb"))
# non_zero_flows = pickle.load(open("data/non_zero_flows.p", "rb"))
# feasible_combinations = pickle.load(open('data/feasible_comb.p', 'rb'))
# optimal_flows = pickle.load(open('data/optimal_flows.p', 'rb'))
# path_lengths = pickle.load(open('data/path_lengths_ship_specific_routes.p', 'rb'))
agent_data = pickle.load(open('data/agent_data.p', 'rb'))
model_data = pickle.load(open('data/model_data.p', 'rb'))

## Process agent data

### Vessel data

In [16]:
agent_data = agent_data.reset_index()

In [17]:
agent_data = agent_data.loc[agent_data.Step>(24*60)]

In [18]:
vessel_data = agent_data.loc[agent_data.vessel_status.notnull()]
vessel_data = vessel_data.drop(columns=['removed_at'])
vessel_data['status'] = 0
vessel_data['at_station'] = 0
vessel_data.status = vessel_data.vessel_status.apply(lambda x: x[0] if type(x) == list else x)
vessel_data.at_station = vessel_data.vessel_status.apply(lambda x: x[1] if type(x) == list else x)

In [19]:
vessel_data_dict = {}
def process_vessel_data(x):
    if not x.AgentID in vessel_data_dict.keys():
        vessel_data_dict[x.AgentID] = {"charging":{}, "inline":{}, "driving":0}
    if x.status == 'driving':
        vessel_data_dict[x.AgentID]['driving'] += 1
    else:
        if not x.at_station in vessel_data_dict[x.AgentID][x.status].keys():
            vessel_data_dict[x.AgentID][x.status][x.at_station] = 1
        else:
            vessel_data_dict[x.AgentID][x.status][x.at_station] += 1

In [20]:
vessel_data.apply(lambda x: process_vessel_data(x), axis=1)

372349     None
372350     None
372351     None
372352     None
372353     None
           ... 
3058376    None
3058377    None
3058378    None
3058379    None
3058380    None
Length: 236835, dtype: object

In [21]:
df = pd.DataFrame(vessel_data_dict)
df = df.transpose()

In [22]:
df['total_charging'] = 0
df.total_charging = df.charging.apply(lambda x: sum(x.values()))
df['total_inline'] = 0
df.total_inline = df.inline.apply(lambda x: sum(x.values()))

In [23]:
df1 = vessel_data.loc[vessel_data.status != 0].loc[:,['AgentID','Step','vessel_route','battery_size', 'generated_at']].groupby('AgentID').first()
vessel_data = pd.merge(df1, df, left_index=True, right_index=True)

In [25]:
vessel_data.loc[vessel_data.generated_at > (24*60)]
vessel_data
# looks okay! Final check with pycharm later

,Step,vessel_route,battery_size,generated_at,charging,inline,driving,total_charging,total_inline
AgentID,,,,,,,,,
44,1441,"(NLAMS, NLHEJ, 0)",1000.0,648.0,{},{},100,0,0
45,1441,"(NLIJM, NLMOE, 1)",1450.0,673.0,{},{},1,0,0
46,1441,"(NLAMS, NLHEJ, 0)",1000.0,714.0,{'8864837': 23},{},202,23,0
63,1441,"(NLNIJ, NLABL, 0)",4750.0,918.0,"{'116': 65, '160': 101}",{},258,166,0
72,1441,"(NLRTM, NLTNZ, 0)",4750.0,1067.0,"{'138': 31, '112': 93, '8866686': 104}",{},407,228,0
...,...,...,...,...,...,...,...,...,...
882,11506,"(NLRTM, NLAPN, 0)",2300.0,11505.0,{},{},15,0,0
883,11508,"(NLSWI, NLLEY, 0)",1450.0,11507.0,{},{},13,0,0
884,11508,"(NLUTC, NLTIE, 0)",4750.0,11507.0,{},{},13,0,0


### Charging station data

In [26]:
df1 = pd.DataFrame(agent_data.loc[(agent_data.charging_stations>0)].groupby('AgentID').first().loc[:,'charging_stations'])

In [27]:
df2 = pd.DataFrame(agent_data.loc[(agent_data.charging_stations>0)].groupby('AgentID').station_status.mean())

In [28]:
cs_data = pd.merge(df1, df2, left_index=True, right_index=True)

In [29]:
cs_data
# TODO check why some are not visited at all within 7 days, seems off

,charging_stations,station_status
AgentID,,
112,1.0,0.363393
113,1.0,0.475992
116,1.0,0.236508
122,1.0,0.189385
124,1.0,0.009821
125,1.0,0.065278
132,1.0,0.085714
134,1.0,0.117560
137,1.0,0.326885


### Remark
Something seems off, a lot of stations are not used at all
Way forward:
1. Check whether this may be clarified based on input data
2. Recheck logic behind generation
3. Check whether the generation is going well and as expected